In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Online Shoppers Intention Dataset

- Target Variable: Revenue.
This indicated whether a session resulted in a purchase (True or False)
- Categorical Variables: Month, VisitorType, Weekend
- Quantitative Variables: Administrative, Administrative_Duration, Informational, ProductRelated, BounceRates, ExitRates, PageValues

### Answering Questions in Project Description:

1. Which variables are predictive of the target variable?
    -  Month, VisitorType, Weekend, Administrative, Administrative_Duration, Informational, ProductRelated, BounceRates, ExitRates, PageValues are predictive of the target variable Revenue

2. Can logistic regression or a linear SVM predict well?
    - Both models can perform well on binary classification tasks. Logistic regression is straightforward and interpretable, while SVMs can handle non-linear boundaries if the data is transformed appropriately. 

3. What do plots of selected pairs of variables look like? Where is the decision boundary in those plots?
    - TBD

4. Is there a difference between the polynomial and RBF SVMs?
    - TBD

5. What effect does changing the class_weight in an SVM have on your data? How might this be important for this data?
    - TBD

6. Is there a difference in runtime performance?
    - TBD

7. Logistic regression and LinearSVC use one-vs-rest (OVR) for multi-class classification. SVC uses one-vs-one (OVO). Where n is the number of classes, OVR learns n models, whereas OVO learns n(n-1)/2 (n choose 2) models. What effect does this have on performance?
    - TBD

In [ ]:
data = pd.read_csv('online_shoppers_intention.csv')
